# Extracting Instruction

It demonstrates a workability of our approach to extract action-instructions from text. For each test document there is a set of names of extracted action and for each action there is a sentence involving the action.

All documents was taken from [this google-disk](https://drive.google.com/drive/folders/1JUv5yHlHYR6hHf3d6MhVKiFjo5Zc4hEN). There are original documents names and links to them below.

In [2]:
import sys
sys.path.append('../../../ScriptExtract')
from TextProcessing import graph_construct as GC
from TextProcessing import research_list, show

## 1. File 'Инструкция по покупке подержанного авто для «продвинутого чайника».txt'

The original is [Инструкция по покупке подержанного авто для «продвинутого чайника»](https://docs.google.com/document/d/1AkoVHz2eJh4xmADGxkzjmS4slSN-Lbpd8xPwfjLBcrE/edit).

### 1.1. Programm Result

In [7]:
handle = open("../../../Texts/text2_2.txt", "r")
text = handle.read()
handle.close()

list_ = GC(text).get_list()  

In [37]:
from TextProcessing import research_action_tree as RAT
import action
def transform(sentences, instr_sentence):
    cur_list = list()
    for sentence in sentences:
        if sentence in instr_sentence:
            cur_list.append((1, sentence))
        else:
            cur_list.append((0, sentence))
    return cur_list
def cur_transform(list_):
    new_list = list()
    for i in list_:
        if not i.__contains__('Type') or i['Type'] == 'paragraph':
            _ = list()
            for j in i['Action tree']:
                _ = _ + RAT(j)
            instr_sentence = [act.sentence for act in _]
            _ = transform([_.sentence for _ in action.construct_tree(i['Text'])], instr_sentence)
            item = list()
            S = [-1] + i['Sentences']
            for ind, j in enumerate(_):
                cur = (j[0], i['Text'][(S[ind]+1):(S[ind+1]+1)])
                item.append(cur)
        else:
            item = cur_transform(i['List'])
        new_list.append(item)
    return new_list

In [38]:
new = cur_transform(list_)

In [39]:
for i in new:
    for j in i:
        if type(j) == type(list()):
            print(j)

In [40]:
new

[[(0, '\ufeffКак выбрать автомобиль с пробегом.')],
 [(0, '4 простых действия при покупке авто.')],
 [(0,
   'Существует великое множество статей о том, как купить автомобиль с пробегом и при этом не «попасть».'),
  (1,
   ' В данной статье я попытаюсь предоставить вам примерный алгоритм действий, которые необходимо будет провести при этом мероприятии, а так же попытаюсь донести кое-какие основы "этикета покупателя", если вообще таковой имеется, дабы не довести продавца "до ручки" глупыми вопросами и действиями=)\n')],
 [(1,
   'Итак, после того как вы определились, какую марку и модель вы хотите приобрести, начинаем поиск.'),
  (0, ' Проще всего в 21-м веке воспользоваться услугами интернет ресурсов.'),
  (1,
   ' Отмечу, что машину стоит искать на авторитетных сайтах таких как: auto.'),
  (0, 'ru, avito.'),
  (0, 'ru, drom.'),
  (0, 'ru и прочие.')],
 [(0,
   'Вторым пунктом, конечно же, телефонный звонок продавцу и несколько простых вопросов, задав которые вы существенно сэкономите 

## 2. Experiments With Semantic

In [41]:
import os
import numpy as np
import time
import pickle
from scipy.linalg import block_diag


In [42]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('../../model.bin', binary=True) 
model.init_sims(replace=True)

/home/ased/anaconda3/lib/python3.5/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [43]:
import sys
sys.path.append('../../')
import sem_analysis as sa


Loading the model...
Processing input...


In [46]:
newest = list()
print(len(new))
ind =0
for i in new:
    _ = list()
    for j in i:
        if type(j) == type(tuple()):
            item = (j[0], sa.tag_ud(j[1]))
        else:
            item = list()
            for k in j:
                item.append((k[0], sa.tag_ud(k[1])))
        _.append(item)
    print(ind)
    ind+=1
    newest.append(_)

9
0
1
2
3
4
5
6
7
8


In [55]:
full_list = list()
for i in newest:
    cur_list = [j for j in i]
    if len(cur_list) >0 and type(cur_list[0]) != type(tuple()):
        _ = list()
        for j in cur_list:
            _ = _ + j
        cur_list = _
    full_list = full_list + (cur_list)

In [69]:
sentences = list()
for i in new:
    cur_list = [j for j in i]
    if len(cur_list) >0 and type(cur_list[0]) != type(tuple()):
        _ = list()
        for j in cur_list:
            _ = _ + j
        cur_list = _
    sentences = sentences + (cur_list)

In [72]:
eps = 1.2
for ind, i in enumerate(full_list):
    if i[0] == 1:
        list_next, list_prev = get_lists(full_list, ind)
        ind_prev = ind - 1
        while ind_prev >-1 and model.wmdistance(full_list[ind_prev][1], i[1]) < eps:
            ind_prev -= 1
        ind_prev += 1
        ind_next = ind + 1
        while ind_next < len(full_list) and model.wmdistance(full_list[ind_next][1], i[1]) < eps:
            ind_next += 1
        ind_next -= 1
        cur_text = ''
        for j in range(ind_prev, ind_next + 1):
            cur_text = cur_text + sentences[j][1]
        print(cur_text, ind_next+1 - ind_prev, '\n'*2)

4 простых действия при покупке авто.Существует великое множество статей о том, как купить автомобиль с пробегом и при этом не «попасть». В данной статье я попытаюсь предоставить вам примерный алгоритм действий, которые необходимо будет провести при этом мероприятии, а так же попытаюсь донести кое-какие основы "этикета покупателя", если вообще таковой имеется, дабы не довести продавца "до ручки" глупыми вопросами и действиями=)
 3 


Итак, после того как вы определились, какую марку и модель вы хотите приобрести, начинаем поиск. 1 


 Отмечу, что машину стоит искать на авторитетных сайтах таких как: auto. 1 


Вторым пунктом, конечно же, телефонный звонок продавцу и несколько простых вопросов, задав которые вы существенно сэкономите своё время. Поинтересоваться, собственник ли он данного авто или его представитель, имеет ли он право продавать данный автомобиль. Является ли авто в «залоге» у банка, наложены ли какие-либо другие финансовые ограничения на данный авто. Далее так же стоит по

In [76]:
def get_center(sentence, model = model):
    s = np.zeros(300)
    k = 0
    for i in sentence:
        if model.__contains__(i):
            s = s + model[i]
            k+=1
    return s/k
full_list_centers = [(i[0], get_center(i[1])) for i in full_list]

/home/ased/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: RuntimeWarning: invalid value encountered in true_divide


In [84]:
eps = 0.5
def distance(a, b):
    return np.linalg.norm(a - b)
for ind, i in enumerate(full_list_centers):
    if i[0] == 1:
        ind_prev = ind - 1
        while ind_prev >-1 and distance(full_list_centers[ind_prev][1], i[1]) < eps:
            ind_prev -= 1
        ind_prev += 1
        ind_next = ind + 1
        while ind_next < len(full_list) and distance(full_list_centers[ind_next][1], i[1]) < eps:
            ind_next += 1
        ind_next -= 1
        cur_text = ''
        for j in range(ind_prev, ind_next + 1):
            cur_text = cur_text + sentences[j][1]
        print(cur_text, ind_next+1 - ind_prev, '\n'*2)

4 простых действия при покупке авто.Существует великое множество статей о том, как купить автомобиль с пробегом и при этом не «попасть». В данной статье я попытаюсь предоставить вам примерный алгоритм действий, которые необходимо будет провести при этом мероприятии, а так же попытаюсь донести кое-какие основы "этикета покупателя", если вообще таковой имеется, дабы не довести продавца "до ручки" глупыми вопросами и действиями=)
Итак, после того как вы определились, какую марку и модель вы хотите приобрести, начинаем поиск. Проще всего в 21-м веке воспользоваться услугами интернет ресурсов. Отмечу, что машину стоит искать на авторитетных сайтах таких как: auto. 6 


Существует великое множество статей о том, как купить автомобиль с пробегом и при этом не «попасть». В данной статье я попытаюсь предоставить вам примерный алгоритм действий, которые необходимо будет провести при этом мероприятии, а так же попытаюсь донести кое-какие основы "этикета покупателя", если вообще таковой имеется, д

In [86]:
eps = 0.5
def distance(a, b):
    return np.linalg.norm(a - b)
def expansion(ind, full_list, eps = 0.4):
    left, right = ind, ind
    prev = None
    while prev is None or right - left != prev:
        prev = right - left
        dl = distance(full_list_centers[left][1], i[1])
        dr = distance(full_list_centers[right][1], i[1])
        if dl < dr:
            if dl < eps and left > 0:
                    left -= 1
        elif dl == dr and dl < eps:
            if left > 0:
                left -= 1
            if right < len(full_list):
                right += 1
        else:
            if dr < eps and right < len(full_list):
                right += 1
        return left, right
for ind, i in enumerate(full_list_centers):
    if i[0] == 1:
        ind_prev, ind_next = expansion(ind, full_list_centers)
        cur_text = ''
        for j in range(ind_prev, ind_next + 1):
            cur_text = cur_text + sentences[j][1]
        print(cur_text, ind_next+1 - ind_prev, '\n'*2)

Существует великое множество статей о том, как купить автомобиль с пробегом и при этом не «попасть». В данной статье я попытаюсь предоставить вам примерный алгоритм действий, которые необходимо будет провести при этом мероприятии, а так же попытаюсь донести кое-какие основы "этикета покупателя", если вообще таковой имеется, дабы не довести продавца "до ручки" глупыми вопросами и действиями=)
Итак, после того как вы определились, какую марку и модель вы хотите приобрести, начинаем поиск. 3 


 В данной статье я попытаюсь предоставить вам примерный алгоритм действий, которые необходимо будет провести при этом мероприятии, а так же попытаюсь донести кое-какие основы "этикета покупателя", если вообще таковой имеется, дабы не довести продавца "до ручки" глупыми вопросами и действиями=)
Итак, после того как вы определились, какую марку и модель вы хотите приобрести, начинаем поиск. Проще всего в 21-м веке воспользоваться услугами интернет ресурсов. 3 


 Проще всего в 21-м веке воспользовать